# Применяем визуальный энкодер к изображениям
Здесь я взял модель CLIP от OpenAi (2021 года)(на соревновании на ODS по рексисам от Авито тоже как раз CLIP использовали для кодирования картинок) и новый энкодер Dino от Meta на удивление при использовании этих эмбеддингов как фичей лучше работает CLIP (метрики на этих фичах лучше)


Однако, есть теория что обложка при покупке книги не является самой важной фичей

**Future work** попробовал бы еще SigLip на конференции WB где рассказывали про SemanticIDs рассказывали что эмбеддинги картинок они делали как раз этой моделью 

## 0 Загружаем данные


In [3]:
# data_folder = "/home/gleb_galagan/tbank_recsys/sirius_recsys/sirius-2025-recsys/data/"
import polars as pl

books = pl.read_parquet('/home/gleb_galagan/tbank_recsys/baseline_solution/books_title_emb.parquet')
print("Books:")
print(books.shape)
display(books.head(5))

Books:
(34322, 9)


item_id,series,tags,title,description,url,image_url,authors,title_embeddings
i64,list[str],list[str],str,str,str,str,list[struct[2]],"array[f32, 1024]"
0,[null],"[""e-book"", ""young-adult"", … ""y-a""]","""Hallie Hath No Fury . . .""","""There are two sides to every s…","""https://www.goodreads.com/book…","""https://images.gr-assets.com/b…","[{""1879494"",""""}]","[-0.014391, -0.001098, … 0.043958]"
1,"[""149079""]","[""primary"", ""melissa-j--morgan"", … ""fiction""]","""Hide and Shriek: Super Special…","""The girls go on an overnight a…","""https://www.goodreads.com/book…","""https://s.gr-assets.com/assets…","[{""21740"",""""}]","[-0.017193, 0.014635, … 0.068515]"
2,[null],"[""friendship"", ""middle-reader"", … ""my-library""]","""Dear Mom, You're Ruining My Li…","""Samantha Slayton worries about…","""https://www.goodreads.com/book…","""https://s.gr-assets.com/assets…","[{""18946"",""""}]","[-0.015779, 0.030699, … 0.000723]"
3,"[""151088""]","[""summer-2017"", ""bullying"", … ""re-read""]","""Bratfest at Tiffany's (Clique …","""Massie Block: The Briarwood bo…","""https://www.goodreads.com/book…","""https://images.gr-assets.com/b…","[{""4605"",""""}]","[-0.034687, 0.014636, … 0.054508]"
4,"[""812067""]","[""rosemary-vernon"", ""young-adult"", … ""to-read""]","""Questions of Love (Sweet Dream…","""When Sammi Edwards is chosen t…","""https://www.goodreads.com/book…","""https://images.gr-assets.com/b…","[{""792676"",""""}]","[-0.027176, 0.023453, … 0.020856]"


## 1 CLIP Embeddings

In [16]:
# Install required libraries
# !pip install polars torch transformers Pillow requests

import polars as pl
import requests
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import torch

# Load a pre-trained model and its processor from Hugging Face
# 'openai/clip-vit-base-patch32' is a popular choice for generating image embeddings.
MODEL_NAME = "openai/clip-vit-base-patch32"

print(f"Loading model: {MODEL_NAME}")
model = CLIPModel.from_pretrained(MODEL_NAME)
processor = CLIPProcessor.from_pretrained(MODEL_NAME)
print("Model loaded successfully.")

# Move model to GPU if available for faster processing
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

Loading model: openai/clip-vit-base-patch32


Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00, 7449.92it/s]


Model loaded successfully.


CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e-05,

## 2 Dino Embeddings

In [ ]:
# Install required libraries if you haven't already
# !pip install polars torch transformers Pillow requests tenacity

import requests
import torch
from PIL import Image
from tenacity import retry, stop_after_attempt, wait_exponential
from transformers import AutoImageProcessor, AutoModel

# --- 1. Model and Processor Configuration ---
# Set the model name to a compatible DINOv2 model
MODEL_NAME = "facebook/dinov3-convnext-small-pretrain-lvd1689m"

print(f"Loading model: {MODEL_NAME}")
# Load the model and its corresponding image processor
model = AutoModel.from_pretrained(MODEL_NAME)
processor = AutoImageProcessor.from_pretrained(MODEL_NAME)
print("Model loaded successfully.")

# Move model to GPU if available for faster processing
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
print(f"Using device: {device}")

# # --- 2. Robust Image Downloading Setup ---
# Create a single session object to reuse connections
session = requests.Session()
# Add a User-Agent header to mimic a web browser
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
session.headers.update(headers)

@retry(
    stop=stop_after_attempt(3),  # Try a maximum of 3 times
    wait=wait_exponential(multiplier=1, min=2, max=10)  # Wait 2s, then 4s, etc.
)
def download_image_with_retries(url: str):
    """A robust function to download an image, separated for clarity."""
    response = session.get(url, stream=True, timeout=15)
    response.raise_for_status()
    return Image.open(response.raw).convert("RGB")

# --- 3. Embedding Generation Function ---
def get_image_embedding(url: str) -> list[float] | None:
    """
    Downloads an image from a URL and returns its DINOv2 embedding.
    Returns None if the image cannot be processed.
    """
    if not url:
        return None
    try:
        # Use the robust download function
        image = download_image_with_retries(url)
        
        # Process the image and generate the embedding
        with torch.no_grad():
            inputs = processor(images=image, return_tensors="pt").to(device)
            
            # Get model outputs
            outputs = model(**inputs)
            
            # The embedding is typically the 'pooler_output' or last hidden state for DINOv2
            embedding = outputs.last_hidden_state[:, 0, :]  # CLS token embedding
            
        print(f'Success: {url}')
        return embedding.cpu().numpy().flatten().tolist()

    except Exception as e:
        print(f"Warning: Failed to get embedding for {url} after multiple retries. Error: {e}")
        return None



Loading model: facebook/dinov3-convnext-small-pretrain-lvd1689m


Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00, 14513.16it/s]

Model loaded successfully.
Using device: cpu


In [5]:
import requests
from tenacity import retry, stop_after_attempt, wait_exponential
from PIL import Image
import torch

# --- All other setup code (loading model, etc.) remains the same ---

# Create a single session object to reuse connections
# This is more efficient than creating a new connection for every request.
session = requests.Session()

# Add a User-Agent header to mimic a web browser
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
session.headers.update(headers)

@retry(
    stop=stop_after_attempt(3),  # Try a maximum of 3 times
    wait=wait_exponential(multiplier=1, min=2, max=10) # Wait 2s, then 4s, etc.
)
def download_image_with_retries(url: str):
    """A robust function to download an image, separated for clarity."""
    response = session.get(url, stream=True, timeout=15)
    response.raise_for_status()
    return Image.open(response.raw).convert("RGB")


def get_image_embedding(url: str) -> list[float] | None:
    """
    Downloads an image from a URL using retries and returns its embedding.
    Returns None if the image cannot be processed after all retries.
    """
    if not url:
        return None
    try:
        # Use the new robust download function
        image = download_image_with_retries(url)
        
        # Process the image and generate the embedding
        with torch.no_grad():
            inputs = processor(images=image, return_tensors="pt").to(device)
            embedding = model.get_image_features(**inputs)
        print(f'{url} Sucess')
        return embedding.cpu().numpy().flatten().tolist()

    except Exception as e:
        # This will now only catch errors that persist after 3 retries
        print(f"Warning: Failed to get embedding for {url} after multiple retries. Error: {e}")
        return None

In [ ]:

# Apply the function to the 'image_url' column to create the new embedding column
df_with_embeddings = books.with_columns(
    pl.col("image_url")
    .map_elements(get_image_embedding, return_dtype=pl.List(pl.Float64))
    .alias("image_dino_embedding")
)

# Display the result
print(df_with_embeddings)

Success: https://images.gr-assets.com/books/1420945127m/23941349.jpg
Success: https://s.gr-assets.com/assets/nophoto/book/111x148-bcc042a9c91a29c1d680899eff700a03.png
Success: https://s.gr-assets.com/assets/nophoto/book/111x148-bcc042a9c91a29c1d680899eff700a03.png
Success: https://images.gr-assets.com/books/1377631907m/7301255.jpg
Success: https://images.gr-assets.com/books/1298284545m/3325990.jpg
Success: https://images.gr-assets.com/books/1348670072m/16042093.jpg
Success: https://images.gr-assets.com/books/1357860035m/17232241.jpg
Success: https://images.gr-assets.com/books/1438268657m/25922869.jpg
Success: https://s.gr-assets.com/assets/nophoto/book/111x148-bcc042a9c91a29c1d680899eff700a03.png
Success: https://images.gr-assets.com/books/1427560553m/25236421.jpg
Success: https://s.gr-assets.com/assets/nophoto/book/111x148-bcc042a9c91a29c1d680899eff700a03.png
Success: https://images.gr-assets.com/books/1331137777m/13515822.jpg
Success: https://images.gr-assets.com/books/1327975077m/13

/home/gleb_galagan/tbank_recsys/base_venv/lib/python3.10/site-packages/PIL/TiffImagePlugin.py:950: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


Success: https://images.gr-assets.com/books/1348358678m/529294.jpg
Success: https://images.gr-assets.com/books/1481269938m/33296824.jpg
Success: https://images.gr-assets.com/books/1441179811m/1605527.jpg
Success: https://s.gr-assets.com/assets/nophoto/book/111x148-bcc042a9c91a29c1d680899eff700a03.png
Success: https://s.gr-assets.com/assets/nophoto/book/111x148-bcc042a9c91a29c1d680899eff700a03.png
Success: https://images.gr-assets.com/books/1256587059m/493568.jpg
Success: https://s.gr-assets.com/assets/nophoto/book/111x148-bcc042a9c91a29c1d680899eff700a03.png
Success: https://images.gr-assets.com/books/1401387779m/22350873.jpg
Success: https://images.gr-assets.com/books/1459988535m/475421.jpg
Success: https://images.gr-assets.com/books/1501165956m/35837982.jpg
Success: https://images.gr-assets.com/books/1382928794m/18510457.jpg
Success: https://s.gr-assets.com/assets/nophoto/book/111x148-bcc042a9c91a29c1d680899eff700a03.png
Success: https://images.gr-assets.com/books/1355098183m/1630034

In [9]:
df_with_embeddings

item_id,series,tags,title,description,url,image_url,authors,title_embeddings,image_dino_embedding
i64,list[str],list[str],str,str,str,str,list[struct[2]],"array[f32, 1024]",list[f64]
0,[null],"[""e-book"", ""young-adult"", … ""y-a""]","""Hallie Hath No Fury . . .""","""There are two sides to every s…","""https://www.goodreads.com/book…","""https://images.gr-assets.com/b…","[{""1879494"",""""}]","[-0.014391, -0.001098, … 0.043958]","[1.309153, 2.461349, … 0.70213]"
1,"[""149079""]","[""primary"", ""melissa-j--morgan"", … ""fiction""]","""Hide and Shriek: Super Special…","""The girls go on an overnight a…","""https://www.goodreads.com/book…","""https://s.gr-assets.com/assets…","[{""21740"",""""}]","[-0.017193, 0.014635, … 0.068515]","[-2.176348, -0.318097, … -2.306087]"
2,[null],"[""friendship"", ""middle-reader"", … ""my-library""]","""Dear Mom, You're Ruining My Li…","""Samantha Slayton worries about…","""https://www.goodreads.com/book…","""https://s.gr-assets.com/assets…","[{""18946"",""""}]","[-0.015779, 0.030699, … 0.000723]","[-2.176348, -0.318097, … -2.306087]"
3,"[""151088""]","[""summer-2017"", ""bullying"", … ""re-read""]","""Bratfest at Tiffany's (Clique …","""Massie Block: The Briarwood bo…","""https://www.goodreads.com/book…","""https://images.gr-assets.com/b…","[{""4605"",""""}]","[-0.034687, 0.014636, … 0.054508]","[-4.33533, 2.080535, … -0.911992]"
4,"[""812067""]","[""rosemary-vernon"", ""young-adult"", … ""to-read""]","""Questions of Love (Sweet Dream…","""When Sammi Edwards is chosen t…","""https://www.goodreads.com/book…","""https://images.gr-assets.com/b…","[{""792676"",""""}]","[-0.027176, 0.023453, … 0.020856]","[-2.626147, -0.692667, … -0.558276]"
…,…,…,…,…,…,…,…,…,…
34317,"[""849421""]","[""will-never-read"", ""action"", … ""fiction""]","""The Cruelty (The Cruelty, #1)""","""When her diplomat father is ki…","""https://www.goodreads.com/book…","""https://images.gr-assets.com/b…","[{""14713542"",""""}]","[-0.025789, 0.045962, … -0.005911]","[0.761839, -2.793995, … -3.114658]"
34318,"[""795937""]","[""haven-t-bothered-to-finish"", ""library"", … ""favorites""]","""Does Snogging Count as Exercis…","""As one of the only girls in Ye…","""https://www.goodreads.com/book…","""https://s.gr-assets.com/assets…","[{""753459"",""""}]","[-0.032484, -0.004517, … 0.019637]","[-2.176348, -0.318097, … -2.306087]"
34319,"[""921523""]","[""young-adult"", ""200-299-pgs"", … ""i-don-t-know""]","""Discovery (The Gifted, #1)""","""All twenty-three year old Kate…","""https://www.goodreads.com/book…","""https://images.gr-assets.com/b…","[{""14590184"",""""}]","[0.007021, 0.001272, … 0.027752]","[-3.493706, 1.530744, … -1.948498]"


## 4 Сохраняем в parquet

In [10]:
df_with_embeddings.write_parquet('books_image_dino_emb.parquet')
